# Fuzzy Run Challenge


In [ ]:
import pandas as pd
import seaborn as sns

## Load data

In [ ]:
df = pd.read_csv("../data/matt.csv")
# df = pd.read_csv("../data/misha.csv")
# df = pd.read_csv("../data/tom.csv")

## Plot acceleration

In [ ]:
df.loc[:, ["aX", "aY", "aZ"]].plot()

## Plot shor segment

In [ ]:
df.loc[6000:6200, ["aX", "aY", "aZ"]].plot()